In [121]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

In [122]:
# waitlist and enrollment data
wl=pd.read_csv("onlywaitlists.csv")
wl

,Class,Data,BCCR1,BCCR2,BCCR3,BCCR4,CCCOMA3,CCCOMA4,CCCOMS3,CCCOMS4,...,GOSTAT2,GSCSCA3,GSCSCA4,GSFNEC4,GSUNDC2,GSUNDC3,PHBION1,OTHER,Total,Cap
0,TOTALS,NaN,132,131,106,89,9,10,103,108,...,0,62,109,5,35,42,0,NaN,NaN,NaN
1,CBMFW4761,enrollment,1,1,1,2,0,0,3,0,...,0,0,4,0,0,0,0,14.0,48.0,50.0
2,CBMFW4761,waitlist,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
3,COMSE6111,enrollment,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1.0,75.0
4,COMSE6111,waitlist,0,0,0,3,0,0,5,12,...,0,2,5,0,0,0,0,5.0,172.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,CSEEW4840,waitlist,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0.0,46.0,NaN
83,CSORW4231,enrollment,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,200.0
84,CSORW4231,waitlist,0,0,7,3,2,0,5,4,...,0,6,7,0,0,0,0,23.0,193.0,NaN
85,ENGIE1006,enrollment,7,9,1,2,0,0,0,1,...,0,0,0,1,3,2,0,22.0,184.0,200.0


In [123]:
# data on what each of the group names mean
st=pd.read_csv( "students.csv")
st

,Program,Program_Name,Degree,Cert_Type,School,Department,Bill_Program_Flag,Interdisciplinary_Flag,Graduation_Flag,Office_Name,Program_Status,Program_Status_Date,Data_Refresh_Date
0,CCCOMA,COMPUTER SCIENCE AND MATHEMATICS,AB,MAJOR,CC,COMS,Y,N,Y,Computer Science,A,12/17/2004 0:00,2/7/2023 5:08
1,CCCOMS,COMPUTER SCIENCE,AB,MAJOR,CC,COMS,Y,N,Y,Computer Science,A,10/1/1991 0:00,2/7/2023 5:08
2,CCCOST,COMPUTER SCIENCE AND STATISTICS,AB,MAJOR,CC,COMS,Y,N,Y,Computer Science,A,9/11/2013 0:00,2/7/2023 5:08
3,CCDTSC,DATA SCIENCE ...,AB,MAJOR,CC,COMS,Y,N,Y,Computer Science,A,9/14/2016 0:00,2/7/2023 5:08
4,CCINFS,INFORMATION SCIENCES,AB,MAJOR,CC,COMS,Y,N,Y,Computer Science,A,6/10/2008 0:00,2/7/2023 5:08
5,CMCOMS,COMPUTER SCIENCE,AB,PREMED CONCN,CC,COMS,Y,N,Y,Computer Science,A,2/28/1997 0:00,2/7/2023 5:08
6,CNCOMS,COMPUTER SCIENCE,AB,CONCENTRATN,CC,COMS,Y,N,Y,Computer Science,A,10/1/1991 0:00,2/7/2023 5:08
7,CS/MBA,DUAL MS/MBA: COMPUTER SCIENCE AND BUSINESS,MS,DEGREE,EP,COMS,Y,N,Y,Computer Science,I,5/6/2000 0:00,2/7/2023 5:08
8,EACSIS,Information Systems,CPF,NON-DEGREE,EP,COMS,Y,N,N,Computer Science,A,5/13/1999 0:00,2/7/2023 5:08
9,EACSIT,Intelligent Systems,CPF,NON-DEGREE,EP,COMS,Y,N,N,Computer Science,A,6/28/1999 0:00,2/7/2023 5:08


In [124]:
# sum together enrollment and waitlist data
wl_data=wl.groupby(['Class']).sum().reset_index()
# dtop the row the totals waitlist number per class
wl_data=wl_data.drop(43)
# drop excess column/rows, transpose to get groups as rows
wl_tr=wl_data.drop(['Class'], axis=1).transpose()
wl_tr=wl_tr.drop(["OTHER", "Total", "Cap"], axis=0)
# compute the total number of waitlists per group
wl_tr["total waitlists"]=wl_tr.sum(axis=1)
# total number of students in each group
wl_tr["total students"]=wl.iloc[0]

# compute the ratio betweem waitlists and total students
# this ratio can be interpeted as the average number of waitlists joined by students in this group
wl_tr["ratio"]=wl_tr["total waitlists"]/wl_tr["total students"]
# divide the number of waitlists per each course by the total number of waitlists
# this can be interpreted as the probability that a student in this group would join the waitlist for this particular class
wl_final=wl_tr[range(0,43)].div(wl_tr["total waitlists"], axis=0)
# bring back the previous columns, fill in naN values with 0
wl_final['waitlist']=wl_tr["total waitlists"]
wl_final['students']=wl_tr["total students"]
wl_final['ratio']=wl_tr["ratio"]
wl_final=wl_final.fillna(0)
wl_final

/var/folders/w5/65l6d67x3qbdqdqfxm1cnkhw0000gn/T/ipykernel_67743/934707582.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wl_data=wl.groupby(['Class']).sum().reset_index()


,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,waitlist,students,ratio
BCCR1,0.005682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.357955,...,0.000000,0.005682,0.000000,0.0,0.000000,0.000000,0.039773,176.0,132,1.333333
BCCR2,0.004167,0.000000,0.000000,0.004167,0.000000,0.000000,0.000000,0.0,0.000000,0.058333,...,0.000000,0.100000,0.000000,0.0,0.000000,0.000000,0.037500,240.0,131,1.832061
BCCR3,0.004587,0.000000,0.004587,0.032110,0.000000,0.000000,0.000000,0.0,0.000000,0.059633,...,0.000000,0.055046,0.022936,0.0,0.000000,0.032110,0.004587,218.0,106,2.056604
BCCR4,0.012048,0.018072,0.012048,0.024096,0.000000,0.006024,0.000000,0.0,0.018072,0.024096,...,0.000000,0.006024,0.018072,0.0,0.000000,0.018072,0.012048,166.0,89,1.865169
CCCOMA3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.076923,0.000000,0.0,0.000000,0.153846,0.000000,13.0,9,1.444444
CCCOMA4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.263158,0.000000,0.0,0.000000,0.000000,0.000000,19.0,10,1.900000
CCCOMS3,0.011070,0.018450,0.000000,0.003690,0.000000,0.003690,0.003690,0.0,0.025830,0.000000,...,0.000000,0.051661,0.029520,0.0,0.000000,0.018450,0.000000,271.0,103,2.631068
CCCOMS4,0.000000,0.054299,0.004525,0.022624,0.000000,0.004525,0.000000,0.0,0.049774,0.000000,...,0.000000,0.018100,0.036199,0.0,0.004525,0.018100,0.004525,221.0,108,2.046296
CCFNEC3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.052632,...,0.000000,0.052632,0.000000,0.0,0.000000,0.052632,0.052632,19.0,11,1.727273
CCNDGV1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0,0.000000


In [125]:
# samples a normal distribution
# need to figure out standard deviation--> could ask for this data? or just use a random number like 1
# sets any negative values to 0
def sample_class_num(mean, std, N):
    sample=np.round(np.random.normal(mean, std, N))
    sample[sample<0]=0
    return sample

# randomly chooses courses according to the distribution
def course_ranking(class_num, course_prob):
    return np.random.choice(43,class_num,p=course_prob, replace=False)

# for a certain group compute a sample of course rankings
def sample_course_ranking(group):
    cr=wl_final.loc[group]
    class_num=sample_class_num(cr.ratio, 1, int(cr.students)).astype(int)
    vec=cr[range(0,43)]
    rank=[course_ranking(x, vec) for x in class_num]
    return rank


In [126]:
sample_class_num(4,1,10)

array([6., 5., 4., 5., 4., 4., 5., 6., 5., 5.])

In [127]:
#SLIGHT BUG HERE, probabilities don't sum to 1
rank=sample_course_ranking("GSUNDC3")
rank

[array([13, 22]),
 array([16]),
 array([28]),
 array([15]),
 array([13,  9]),
 array([28]),
 array([], dtype=int64),
 array([37]),
 array([14, 19, 16]),
 array([9]),
 array([14]),
 array([13]),
 array([15, 13]),
 array([29]),
 array([42, 28]),
 array([15]),
 array([16, 15,  9]),
 array([28]),
 array([28]),
 array([22, 13, 35]),
 array([], dtype=int64),
 array([13]),
 array([14]),
 array([ 9, 16]),
 array([14, 12]),
 array([13, 14]),
 array([], dtype=int64),
 array([13, 15]),
 array([13,  9, 12]),
 array([28, 37, 13]),
 array([13, 15, 28]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([13, 14, 15]),
 array([13]),
 array([14, 28]),
 array([14, 17, 13]),
 array([15, 13, 22, 37,  9]),
 array([15]),
 array([42, 15, 28]),
 array([13, 14])]

In [128]:
## create ranks here
## i.e. [1,2,3]
## schedule with [1] will have score 3
## schedule with [2] will have score 2
## schedule with [3] will have score 1
## utility is sum of scores, sort bundles by highest utility


def generate_demand_set(arr, max_bundle_size):
    possible_bundles = []
    for i in range(len(arr)):
        for j in range(i + 1, len(arr) + 1):
            bundle = arr[i:j]
            possible_bundles.append((sum(bundle), bundle))
    #put in order of utility
    possible_bundles.sort(key=lambda x: (x[0], len(x[1])), reverse=True)
    #get rid of bundles that are too large
    final_possible_bundles = [x[1] for x in possible_bundles if len(x) <= max_bundle_size]
    return final_possible_bundles



In [129]:
rank=sample_course_ranking("GSUNDC3")

all_demand_sets = []
for student in rank:
    demand_set = generate_demand_set(student, 3)
    all_demand_sets.append(demand_set)

all_demand_sets



[[array([37])],
 [],
 [array([13,  9, 14]),
  array([ 9, 14]),
  array([13,  9]),
  array([14]),
  array([13]),
  array([9])],
 [array([19])],
 [array([13])],
 [array([9])],
 [],
 [array([15, 16, 14]),
  array([15, 16]),
  array([16, 14]),
  array([16]),
  array([15]),
  array([14])],
 [array([13, 35]), array([35]), array([13])],
 [],
 [array([37])],
 [],
 [],
 [],
 [array([12, 16]), array([16]), array([12])],
 [array([13, 14,  9]),
  array([13, 14]),
  array([14,  9]),
  array([14]),
  array([13]),
  array([9])],
 [array([37, 13, 16]),
  array([37, 13]),
  array([37]),
  array([13, 16]),
  array([16]),
  array([13])],
 [array([16, 12]), array([16]), array([12])],
 [array([13, 42]), array([42]), array([13])],
 [array([35, 33]), array([35]), array([33])],
 [array([15])],
 [array([13, 28]), array([28]), array([13])],
 [array([13])],
 [array([13])],
 [array([13])],
 [array([12])],
 [array([35, 14, 19]),
  array([35, 14]),
  array([35]),
  array([14, 19]),
  array([19]),
  array([14])],
 [

In [130]:
rank=sample_course_ranking("GSUNDC3")
print(rank)
def get_all_demand_set(rank,max_bundle_size=3):
    all_demand_sets = []
    for student in rank:
        demand_set = generate_demand_set(student, max_bundle_size)
        all_demand_sets.append(demand_set)

    return all_demand_sets

[array([], dtype=int64), array([15]), array([ 9, 35]), array([22]), array([15, 13,  9]), array([13, 37]), array([35]), array([14]), array([13]), array([42, 12, 15]), array([22]), array([15]), array([14, 15, 12]), array([13, 35]), array([13, 35]), array([ 9, 15, 22]), array([13, 14, 16]), array([42, 15, 33]), array([], dtype=int64), array([42, 28]), array([28]), array([ 9, 13, 12]), array([13, 16]), array([15]), array([15, 14]), array([16, 37, 13]), array([14, 13]), array([ 9, 12]), array([17, 13, 16]), array([16]), array([], dtype=int64), array([13, 14]), array([19, 12, 13]), array([14]), array([], dtype=int64), array([13]), array([13, 37, 17]), array([9]), array([15, 37, 14]), array([14, 12]), array([33, 17]), array([37, 13])]


In [131]:
wl.fillna(0, inplace=True)
wl_data2=wl.groupby(['Class']).sum().reset_index()
wl_data2

/var/folders/w5/65l6d67x3qbdqdqfxm1cnkhw0000gn/T/ipykernel_67743/3397330978.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wl_data2=wl.groupby(['Class']).sum().reset_index()


,Class,BCCR1,BCCR2,BCCR3,BCCR4,CCCOMA3,CCCOMA4,CCCOMS3,CCCOMS4,CCFNEC3,...,GOSTAT2,GSCSCA3,GSCSCA4,GSFNEC4,GSUNDC2,GSUNDC3,PHBION1,OTHER,Total,Cap
0,CBMFW4761,1,1,1,2,0,0,3,0,0,...,0,0,4,0,0,0,0,14.0,48.0,50.0
1,COMSE6111,0,0,0,3,0,0,5,12,0,...,0,2,5,0,0,0,0,5.0,173.0,75.0
2,COMSE6156,0,0,1,2,0,0,0,1,0,...,0,0,3,0,0,0,0,0.0,19.0,30.0
3,COMSE6178,0,1,7,4,0,0,1,5,0,...,0,0,2,0,0,0,0,8.0,71.0,30.0
4,COMSE6261,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,2.0,12.0,35.0
5,COMSE6424,0,0,0,1,0,0,1,1,0,...,0,0,2,0,0,0,0,0.0,38.0,30.0
6,COMSE6732,0,0,0,0,0,0,1,0,0,...,0,0,2,0,0,0,0,0.0,21.0,20.0
7,COMSE6915,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,25.0
8,COMSE6998,0,0,0,3,0,0,7,11,0,...,0,2,6,0,0,0,0,8.0,235.0,490.0
9,COMSW1004,63,14,13,4,0,0,0,0,1,...,0,0,0,0,10,8,0,34.0,267.0,796.0


In [132]:
# Creates a dataframe of 1's and 0's for the specified group
# based on the demand set that provides the highest 
# utility (i.e. the first set  returned from get_all_demand_set(rank, bundle_size) 
def allocate_group(group_demand_sets):
    max_demand_set=[]
    for i,demand in enumerate(group_demand_sets):
        if len(demand) != 0:
            max_demand_set.append(demand[0])
            
    rows = len(wl_data2)
    columns = len(max_demand_set)
    matrix_of_zeros = np.zeros((rows, columns))

    df_allocation=pd.DataFrame(matrix_of_zeros)
    df_allocation['Class']=wl_data2['Class']
    last_column = df_allocation.columns[-1]

    # Move the last column to the first position
    df_allocation = df_allocation[[last_column] + [col for col in df_allocation.columns if col != last_column]]

    for i,classes in enumerate(max_demand_set):
        j=0
        while j <len(classes):
            df_allocation.iloc[classes[j],i]=1
            j+=1
            
            
    return df_allocation

In [133]:
def one_hot_encode(input_list):
    encoded_vector = np.zeros(43)
    encoded_vector[input_list] = 1
    return encoded_vector.tolist()

def demand_set_to_one_hot(all_demand_set):
	one_hot_demand_sets = []
	for student_ds in all_demand_set: 
		student_one_hot_ds = []
		for bundle in student_ds: 
			one_hot = one_hot_encode(bundle)
			student_one_hot_ds.append(one_hot)
		one_hot_demand_sets.append(student_one_hot_ds)
	return one_hot_demand_sets

def one_hot_to_demand_set(all_one_hot_demand_sets):
	demand_sets = []
	for student_one_hot_ds in all_one_hot_demand_sets:
		student_demand_set = []
		for bundle in student_one_hot_ds: 
			student_demand_set.append([index for index, value in enumerate(bundle) if value == 1])
		demand_sets.append(student_demand_set)
	return demand_sets 


In [134]:
def get_prices_for_each_allocation(prices, student_demand_set):
	price_for_each_demand_set = []
	for demand_set in student_demand_set: 
		prices_np = np.array(prices)
		demand_set_np = np.array(demand_set).T
		total_price = prices_np.dot(demand_set_np)
		price_for_each_demand_set.append(total_price[0])
	
	return price_for_each_demand_set

def budget_perterbations(prices, student_demand_set): 
	# input: prices for each course & demand set for a student 
	# prices = [.3, .2, .1, .5] (ie, price of course at idx 0 is 0.3, price of course at idx 1 is 0.2)
	# student_demand_set = [[1,1,0,0], [1,0,0,0],[0,1,0,0]] where a student's most preferred bundle is [1,1,0,0] and they get classes 0 and 1
	# second preferred bundle is [1,0,0,0] where the student would get only class 0, etc. 

	# output: budget allocation for each student, ie if they spent 0.2 to 0.5 (noninclusive), they would want allocation [1,0,0,0]
	# if they spent 0.5 or more, they would want allocation [1,1,0,0]

	#idx 0 price is for the top ranked bundle 
	if len(student_demand_set) == 0:
		return ([],[])
	
	price_for_each_demand_set = get_prices_for_each_allocation(prices,student_demand_set)
	allocations = []
	budget_intervals = []
	last_min_price = max(price_for_each_demand_set) + 1

	for i, bundle_price in enumerate(price_for_each_demand_set): 
		if bundle_price < last_min_price: 
			if len(student_demand_set[i]) == 0:
				continue
			allocations.append(student_demand_set[i])
			budget_intervals.append((bundle_price, last_min_price))
			last_min_price = bundle_price
# 	print("alloc: ", allocations)
# 	print("budget intervals: ", budget_intervals)

	return (allocations, budget_intervals)


In [135]:
def pad_arrays(allocations):
	max_len = len(max(allocations, key=len)) 
	for allocation in allocations: 
		while len(allocation) < max_len: 
			allocation.append(allocation[-1])
	return allocations





In [136]:
# step size and error 

def clearing_error_optimizer(a, c, p):
	n=len(a)
	m=len(c)
	ki=len(a[0])
	# Create a new model
	model = gp.Model("Clearing_Error_Minimization")

	# Decision variables
	# These are the dimensions of the decision variable array
	# array of binary decision variables with n rows and ki columns
	z = model.addVars(m, lb=0.0, vtype=GRB.INTEGER, name="z")
	x = model.addVars(n, ki, vtype=GRB.BINARY, name="x")
	print(z)

	# Objective function: Minimize the l1-norm of vector z
	model.setObjective(gp.quicksum(z[j] for j in range(m)), sense=GRB.MINIMIZE)

	# Constraints
	for j in range(m):
		if p[j]>0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) == c[j] + z[j], f"clearing_error_positive_{j}")
		if p[j]==0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) <= c[j] + z[j], f"clearing_error_nonnegative_{j}")
	
	for i in range(n):
		model.addConstr(gp.quicksum(x[i, l] for l in range(ki)) == 1, f"one_schedule_per_student_{i}")

	# Solve the model
	model.optimize()

	# Check optimization status
	if model.status == GRB.OPTIMAL:
		# Access the optimal solution
		optimal_x = model.getAttr("x", x)
		optimal_z = model.getAttr("x", z)

		# Print or use the optimal solution as needed
		print("Optimal x values:", optimal_x)
		print("Optimal z values:", optimal_z)
		return (optimal_x, optimal_z)
	else:
		print(f"Optimization terminated with status {model.status}")
		# Check if the model is infeasible
		print(f"HELLO")
		model.computeIIS()
		print("\nInfeasible constraints:")
		for c in model.getConstrs():
			if c.IISConstr:
				print(c.constrName)
		return (None, None)




In [137]:
#example input
def show_classes_per_student(x, bundles): 
	student_classes = {}
	for key, value in x.items(): 
		bundle = bundles[key[0]][key[1]]
		if value != 0: 
			if key[0] not in student_classes: 
				student_classes[key[0]] = bundle

	return student_classes

def show_students_per_class(x, bundles):
	class_students = {}
	for key, value in x.items(): 
		if value == 0:
			continue
		bundle = bundles[key[0]][key[1]]
		for i, class_num in enumerate(bundle):
			if class_num == 1 and i not in class_students: 
				class_students[i] = [key[0]]
			elif class_num == 1: 
				class_students[i].append(key[0])
	return class_students

def get_budget_perterbations(prices, one_hot_demand_set):
	all_budget_perterbations = []
	all_budget_intervals = []
	for student_demand_set in one_hot_demand_set: 
		allocations, budget_intervals = budget_perterbations(prices, student_demand_set)
		if allocations != [] and budget_intervals != []:
			all_budget_perterbations.append(allocations)
			all_budget_intervals.append(budget_intervals)

	allocations = pad_arrays(all_budget_perterbations)
	return allocations

def pretty_print_output(x, one_hot_demand_set, prices, f):
	student_classes = show_classes_per_student(x, one_hot_demand_set)
	for key in student_classes: 
		f.write("student: "+str(key)+ " classes: "+str(student_classes[key]) + "\n")

	classes_per_student = show_students_per_class(x, one_hot_demand_set)
	for key in classes_per_student: 
		f.write("class: "+ str(key)+" students: "+ str(classes_per_student[key])+ "\n")
	string_list = [str(float_value) for float_value in prices[0]]
	prices_string = ' '.join(string_list)
	f.write("prices: "+ prices_string+ "\n")

def aceei(one_hot_demand_set): 
	# rank=sample_course_ranking("CCCOMS3")
	# all_demand_set=get_all_demand_set(rank,3)
	# one_hot_demand_set = demand_set_to_one_hot(all_demand_set)
	f = open("out.txt", "w+")

	print(one_hot_demand_set)
	prices = [[0,0]]
	for i in range(3): 
		f.write("\n Epoch: " + str(i) + "\n")
		a = get_budget_perterbations(prices, one_hot_demand_set)
		c = [3,2]
		p = prices[0]
		x, z = clearing_error_optimizer(a, c, p)
		pretty_print_output(x, one_hot_demand_set, prices, f)
		prices = (np.array(prices) + np.array(list(z.values()))*0.01).tolist() 

	f.close()
	return (prices, x)


In [138]:


one_hot_demand_set = [[[1,1], [0,1], [1,0]], [[1,1], [0,1], [1,0]], [[1,1], [0,1],[0,1]], [[1,1], [1,0], [0,1]]]

prices, x = aceei(one_hot_demand_set)



[[[1, 1], [0, 1], [1, 0]], [[1, 1], [0, 1], [1, 0]], [[1, 1], [0, 1], [0, 1]], [[1, 1], [1, 0], [0, 1]]]
{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x4f20cee2
Variable types: 0 continuous, 6 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, bes

In [139]:



print(prices)

[[0.01, 0.02]]
